In [1]:
!pip install plotly
!pip install geopy

  Using cached plotly-5.1.0-py2.py3-none-any.whl (20.6 MB)
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
  Using cached geopy-2.2.0-py3-none-any.whl (118 kB)
  Using cached geographiclib-1.52-py3-none-any.whl (38 kB)


In [2]:
import pandas as pd 
import numpy as np
import sklearn
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
import plotly.io as pio
import geopy.distance
import requests


import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
pio.renderers.default = "iframe"

In [3]:
data = pd.read_csv('uber-raw-data-apr14.csv')

In [4]:
display(data.shape)
display(data.dtypes)
display(data.head())
display(data.describe(include='all'))

(113861, 4)

Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


,Date/Time,Lat,Lon,Base
count,113861,113861.000000,113860.000000,113860
unique,29378,NaN,NaN,2
top,4/7/2014 20:21:00,NaN,NaN,B02598
freq,43,NaN,NaN,78324
mean,NaN,40.741480,-73.977672,NaN
std,NaN,0.036362,0.051749,NaN
min,NaN,40.225900,-74.703900,NaN
25%,NaN,40.723400,-73.997800,NaN
50%,NaN,40.744100,-73.984400,NaN
75%,NaN,40.761800,-73.969900,NaN


In [5]:
data['dayofweek'] = pd.to_datetime(data["Date/Time"]).dt.dayofweek
data['hour'] = pd.to_datetime(data["Date/Time"]).dt.hour

coord = requests.get("https://nominatim.openstreetmap.org/?q=<manhattan>&format=json&limit=1").json()[0]
lat = float(coord['lat'])
lon = float(coord['lon'])

In [6]:
mask_lat = ~data.loc[:,'Lat'].isnull()
mask_lon = ~data.loc[:,'Lon'].isnull()

data = data.loc[(mask_lon) & (mask_lat),:]

In [12]:
data["distance_to_center"] = data.apply(lambda x: geopy.distance.distance((x['Lat'],x['Lon']), (lat,lon)).m,axis = 1)

In [13]:
data["distance_to_center"].head()

0    2328.751024
1    9408.283690
2    6846.465854
3    3735.124364
4    3513.459106
Name: distance_to_center, dtype: float64

In [14]:
dataset = data.loc[(data['dayofweek']== 6) & (data['hour']== 12),:]

X =dataset[["distance_to_center"]].values

In [31]:

# Clusqter haute densité
db = DBSCAN(eps=500, min_samples=50
            , metric="manhattan", algorithm="brute")

# Fit on data 
db.fit(X)

# Visualize with plotly 


cluster_haute_densite = db.labels_


# Cluster basse densité
db_basse_densite = DBSCAN(eps=0.015, min_samples=20
            , metric="manhattan", algorithm="brute")

# Fit on data 
db_basse_densite.fit(X)

cluster_basse_densite = db_basse_densite.labels_

dataset["haute_densite"] = cluster_haute_densite !=-1
dataset["basse_densite"] = cluster_basse_densite !=-1

dataset["haute_densite"] = dataset["haute_densite"].astype("int")
dataset["basse_densite"] = dataset["basse_densite"].astype("int")


fig = px.scatter_mapbox(dataset,hover_name =db.labels_,lat='Lat', lon='Lon', color=db.labels_,zoom=10, mapbox_style="carto-positron")
fig.show("iframe")


In [11]:
print(dataset["haute_densite"].shape)
(cluster_haute_densite !=-1).astype("int").shape

(513,)


(513,)

In [12]:
for i in range(24):

    dataset = data.loc[(data['dayofweek']== 6) & (data['hour']== i),:]
    X =dataset[["Lat",'Lon']].values

    # Cluster haute densité
    db = DBSCAN(eps=0.015, min_samples=70, metric="manhattan", algorithm="brute")

    # Fit on data 
    db.fit(X)
    cluster_haute_densite = db.labels_


    # Cluster basse densité
    db_basse_densite = DBSCAN(eps=0.015, min_samples=20, metric="manhattan", algorithm="brute")

    # Fit on data 
    db_basse_densite.fit(X)

    cluster_basse_densite = db_basse_densite.labels_

    data.loc[(data['dayofweek']== 6) & (data['hour']== i),"haute_densite"] = (cluster_haute_densite !=-1).astype("int")
    data.loc[(data['dayofweek']== 6) & (data['hour']== i),"basse_densite"] = (cluster_basse_densite !=-1).astype("int")


In [13]:
data.columns

Index(['Date/Time', 'Lat', 'Lon', 'Base', 'dayofweek', 'hour',
       'distance_to_center', 'haute_densite', 'basse_densite'],
      dtype='object')

In [14]:
fig = px.scatter_mapbox(data, lat="Lat", lon="Lon", color="haute_densite", zoom = 10,
                        mapbox_style="open-street-map", color_continuous_scale = 'Reds', range_color = [5.0,10.0],
                       animation_frame = 'hour')
fig.show("iframe")

In [15]:
fig = px.density_mapbox(earthquakes, lat="Latitude", lon="Longitude", mapbox_style="open-street-map",
                       animation_frame = 'hour', zoom = 10, radius = 10)
fig.show()

NameError: name 'earthquakes' is not defined